In [1]:
%matplotlib inline

import matplotlib 
import numpy as np
import pandas as pd
import os, sys, time, pickle, copy
#import ipywidgets
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection    

from scipy import optimize

import cv2 

matplotlib.rc('font', family='serif')
matplotlib.rc('font', size=10)
matplotlib.rc('legend', fontsize=16)
matplotlib.rc('legend', numpoints=1)
matplotlib.rc('legend', handlelength=1.5)
matplotlib.rc('legend', frameon=True)
matplotlib.rc('xtick.major', pad=7)
matplotlib.rc('xtick', direction="in")
matplotlib.rc('ytick', direction="in")
matplotlib.rc('xtick', top = True)
matplotlib.rc('ytick', right =True )
matplotlib.rc('xtick.minor', pad=7)
matplotlib.rc('text', usetex=True)

matplotlib.rc('figure', figsize=(12, 9))





# Chapter 8. 칼라 이미지 처리 (Color Image Processing)

## 8.1. 개관


## 8.2. 색의 구분

#### 색상 (hue, H), 채도 (saturation, S), 그리고 명 도 (brightness or value, V)  

- 색상은 여러 파장의 빛이 섞여 있을 때 관찰자 입장 에서 가장 눈에 띄는 대표적인 파장으로 나타나는 색을 의미하며, 
- 채도는 색상에 얼마나 높은 비율로 하얀색의 빛 (백색광)이 섞여 있는지를 의미하고 (즉, 하얀색은 채도가 0, 삼 원색이나 2차색 (CMY)은 채도가 1),
- 명도는 색상 정보와 상관 없이 명암 이미지에서 말 하는 픽셀 값 (강도)를 의미합니다. 


#### CIE가 20세기 초반에 했던 실험
- CIE는 인간의 원추 세포가 적어도 단파장, 중파장, 장파장 의 세 가지 빛을 구분할 수 있다는 것을 알고 있었으므로, 원칙적으로 세 개의 변수 만 으로 인간이 인지할 수 있는 색감을 전부 표현할 수 있을 것이라는 가설을 세웠습니다. 
- 이를 기반으로, CIE는 <b>XYZ 색 공간</b>이라는 개념을 도입하였는데, X, Y, Z는 각 파장 (즉, 각 색깔)에 대응하는 <b>삼색 자극값 (tristimulus values)</b>을 의미합니다. 예를 들어, X 빨강, Y는 초록, Z는 파랑과 비슷한 색깔에 대응합니다. 인간의 색상 인지 특성 중 하나는 다양한 파장을 가진 두 가지 다른 색깔을 섞으면, 다른 색과 비슷하게 보일 수 있다는 것입니다. 이러한 현상을 <b>조건등색 (metamerism)</b> 이라고도 하는데, 두 개의 색깔이 가진 삼색 자극 값의 합은 두 개의 색깔이 갖는 파장과 상관 없이, 똑같아 보이는 다른 색의 삼색 자극 값과 동일함을 의미합니다. 

#### 삼색자극갑과 RGB
삼색 자극값 좌표 $\begin{bmatrix} X &Y &Z \end{bmatrix}$ 를 인간이 인지하는 좌표인 $\begin{bmatrix} R & G &B \end{bmatrix}$ 로 바꾸는 것 (혹은 역변환은)은 다음의 행렬 연산 과정을 거치면 됩니다.

$$
\begin{aligned}
\begin{bmatrix} R \\ G \\ B \end{bmatrix} & = \begin{bmatrix} 3.063  & -1.393  & -0.476 \\ -0.969 & 1.876 &  0.042 \\ 0.068  & -0.229 & 1.069 \end{bmatrix} \begin{bmatrix} X \\ Y \\ Z \end{bmatrix} \\
{}\\
\begin {bmatrix} X\\ Y \\ Z \end{bmatrix} &=\begin{bmatrix} 0.431  & 1.342  & 0.178 \\ 0.222 & 0.797 &  0.071 \\ 0.020  & 0.130 & 0.939 \end{bmatrix} \begin{bmatrix} R \\ G \\ B \end{bmatrix}
\end{aligned}
$$

참고로 $C,\,M,\,Y$ 는 다음과 같이 계산된다. (C=청록, M=분홍, Y= 노랑 으로 색소의 삼원색이다.)

$$
\begin{bmatrix} C\\ M \\ Y \end{bmatrix} = \begin{bmatrix} 1\\ 1 \\ 1 \end{bmatrix} - \begin{bmatrix} R\\ G \\ B \end{bmatrix}
$$

#### RGB to HSV

$$
\begin{aligned}
V &= \max(R,G,B),\,\\
\delta &= V -\min (R,G,B) ,\,\\
S &= \dfrac{\delta}{V} \\
\end{aligned}
$$

$$
\begin{aligned}
H &= \left \{ \begin{array}{ll}
\dfrac{G-B}{6\delta} \qquad&\text{when } V=R \\ 
\dfrac{1}{6} \left( 2 + \dfrac{B-R}{\delta} \right) & \text{when } V=G \\ 
\dfrac{1}{6} \left(4 + \dfrac{R-G}{\delta} \right) & \text{when } V=B 
\end{array} 
\right.
\end{aligned}
$$

만약 $H$ 가 음수이면 $1+H$ 값이 대신한다. 또한 다음과 같이 정의한다.
$$
\begin{bmatrix} R & G & B \end{bmatrix} = \begin{bmatrix} 0 & 0 & 0 \end{bmatrix} \implies \begin{bmatrix} H & S & V \end{bmatrix} = \begin{bmatrix} 0 & 0 & 0 \end{bmatrix} 
$$


##### HSV to RGB

$$
\begin{aligned}
H' & =  [6H ], \\
F &= 6H - H', \\
P &= V (1 -S) , \\
Q &= V (1- SF) ,\\
T &= V(1-S(1-F)).
\end{aligned}
$$

여기서 $[6H]$ 는 소위 가우스 기호로 $6H$ 에 가장 가까운 정수값을 의미한다.



In [20]:
def rgb2hsv(r, g, b, dd = 0.01):
    if r<dd and g<dd and b < dd :
        return (0., 0., 0.)
    v = max(r, g, b)
    delta = v-min(r, g, b)
    s = delta/v
    if v == r :
        h = (g-b)/6./delta
    elif v == g :
        h = 1./6*(2+(b-r)/delta)
    elif v == b :
        h=1./6 *(4+ (r-g)/delta)
    if h<0 :
        h = 1-h
    return (h, s, v)

def hsv2rgb(h, s, v, dd=0.01):
    h2 = int(round(6*h))
    f = 6*h - h2
    p = v*(1-s)
    q = v*(1.-s*f)
    t = v*(1-s*(1-f))
    
    if h2 == 0:
        return(v, t, p)
    elif h2 == 1:
        return (q, v, p)
    elif h2 == 2:
        return (p, v, t)
    elif h2 == 3:
        return (p, q, v)
    elif h2 == 4:
        return (t, p, v)
    else :
        return (v, p, q)


In [21]:
print(rgb2hsv(0.2,0.4,0.6))
print(hsv2rgb(0.5833,0.6667,0.6000))

(0.5833333333333333, 0.6666666666666666, 0.6)
(0.19998000000000002, 0.4000700039999998, 0.6)


# 가상 칼라 이미징 (Pseudocoloring)